# 把v2.1标准模型从ckpt版转化成diffusers版

# 安装Transformers

In [ ]:
import subprocess

libraries_to_install = [
    "transformers",
    "accelerate"
]

tools_to_install = [
    # "tree"
    "iputils-ping"
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool
    


# 安装Diffusers

In [ ]:
import os
import subprocess

!pip uninstall -y diffusers

# 调用命令行工具git rev-parse，获取git项目的根目录
git_root = subprocess.run(
    ["git", "rev-parse", "--show-toplevel"],
    capture_output=True,
    text=True,
).stdout.strip()

# 获取根目录的父目录
parent_dir = os.path.dirname(git_root)

content_folder=parent_dir

# 检测 diffusers 目录是否存在
if not os.path.exists(os.path.join(content_folder, 'diffusers')):
    # 如果 diffusers 目录不存在，则进行 clone
    !cd $content_folder &&\
    git clone --branch updt https://github.com/TheLastBen/diffusers
else:
    # 如果 diffusers 目录已经存在，则不进行 clone，直接输出提示信息
    print('diffusers 目录已经存在，不进行 clone 操作。')
  
!pip install --user $content_folder/diffusers
print('[1;32mDONE !')

# 下载Convert To Diff v2和Convert To SD v2的python脚本

In [ ]:
# 导入 os 模块
import os

# 创建临时目录 temp
if not os.path.exists('temp'):
    os.makedirs('temp')

# 下载 .py 脚本并保存到 temp 目录下
!wget -O temp/convertodiff.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/convertodiffv2.py
!wget -O temp/convertosdv2.py https://raw.githubusercontent.com/TheLastBen/diffusers/main/scripts/convertosdv2.py

# 把input0中的diffusers权重转化成ckpt格式的模型

In [ ]:
prc="--fp16"
WEIGHT_DIR='/input0'
CKPT_DIR='/output/v2-1_768-nonema-pruned.ckpt'

import time
from datetime import timedelta

# 脚本开始执行的时间
start_time = time.time()

# 这里是你的脚本代码
!python ./temp/convertosdv2.py $prc $WEIGHT_DIR $CKPT_DIR

# 脚本执行完成的时间
end_time = time.time()
# 脚本执行的总耗时
elapsed_time = end_time - start_time
# 将脚本执行的总耗时转换为时分秒的表示形式
elapsed_time_str = str(timedelta(seconds=elapsed_time))
print("转化总耗时：", elapsed_time_str)

!du -sh $CKPT_DIR

# 把ckpt格式的模型转化为diffusers权重

In [ ]:
WEIGHT_DIR='/output/stable-diffusion-custom'
CKPT_DIR='/output/v2-1_768-nonema-pruned.ckpt'

import time
from datetime import timedelta

# 脚本开始执行的时间
start_time = time.time()

# 这里是你的脚本代码
!python ./temp/convertodiff.py $CKPT_DIR $WEIGHT_DIR --v2 --reference_model stabilityai/stable-diffusion-2-1

# 脚本执行完成的时间
end_time = time.time()
# 脚本执行的总耗时
elapsed_time = end_time - start_time
# 将脚本执行的总耗时转换为时分秒的表示形式
elapsed_time_str = str(timedelta(seconds=elapsed_time))
print("转化总耗时：", elapsed_time_str)

!du -sh $WEIGHT_DIR